In [1]:
import pandas as pd
import numpy as np

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
TARGET_LEAGUES = [
    'LTC', 'LCK',
    'LPL',
    'EU LCS', 'LEC',
    'NA LCS', 'LCS',
    'GPL', 'LST',
    'LLA', 'CLS', 'LLN',
    'LCO', 'OPL',
    'LCL',
    'LJL',
    'LMS', 'PCS',
    'VCS',
    'CBLOL', 
    'TCL',
    
    'MSI',
    'WCS',
]

In [3]:
def get_new_id(ids):
    if 'team_id' in ids.columns:
        id_list = sorted(ids['team_id'].unique())
    else:
        id_list = sorted(ids['player_id'].unique())

    if len(id_list) == 0:
        return 1
    prev = id_list[0]
    for id in id_list[1:]:
        if prev + 1 != id:
            return prev + 1
        prev = id
    return prev + 1

## Teams

In [4]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [5]:
lst = []
for league in TARGET_LEAGUES:
    if league not in leagues['League Short'].values:
        lst.append(league)
lst

[]

In [6]:
teams = pd.read_csv('./csv/teams_id.csv')
names = []

# for league in TARGET_LEAGUES:
for league in leagues['League Short']:
    print(league)
    tournaments = get_tournaments(where=f'L.League_Short="{league}"')
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    for page in tournaments['OverviewPage']:
        print(f'\t{page}')
        try:
            scoreboard_games = get_scoreboard_games(
                where=f'T.OverviewPage="{page}"'
            )
            scoreboard_players = get_scoreboard_players(
                where=f'T.OverviewPage="{page}"'
            )
        except Exception as e:
            print(e)
            continue
        if scoreboard_games is not None:
            scoreboard_games = scoreboard_games.sort_values(
                by='DateTime UTC'
            ).reset_index(drop=True)
            team_names = scoreboard_games[['Team1', 'Team2']].unstack().unique()
            names = []
            for name in team_names:
                if name not in teams['team'].values:
                    names.append(name)
            if len(names) > 0:
                names = sorted(names)
                break
        if scoreboard_players is not None:
            scoreboard_players = scoreboard_players.sort_values(
                by='DateTime UTC'
            ).reset_index(drop=True)
            team_names = scoreboard_players['Team'].unique()
            names = []
            for name in team_names:
                if name not in teams['team'].values:
                    names.append(name)
            if len(names) > 0:
                names = sorted(names)
                break
    if len(names) > 0:
        print(scoreboard_games['OverviewPage'].unique()[0])
        print(scoreboard_players['OverviewPage'].unique()[0])
        print(names)
        break

2015 IWCT
	None
'OverviewPage'
IWCQ
	2016 International Wildcard Qualifier
CBLOL
	CBLOL/2014 Season/Champions Series
	CBLOL/2014 Season/Champions Series/Playoffs
	CBLOL/2015 Season/Split 1 Qualifiers
	CBLOL/2015 Season/Split 1
	CBLOL/2015 Season/Split 1 Playoffs
	CBLOL/2015 Season/Split 2 Promotion
	CBLOL/2015 Season/Split 2
	CBLOL/2015 Season/Split 2 Playoffs
	CBLOL/2015 Season/Post-Season
	CBLOL/2016 Season/Split 1 Promotion
	CBLOL/2016 Season/Split 1
	CBLOL/2016 Season/Split 1 Playoffs
	CBLOL/2016 Season/Split 2 Promotion
	CBLOL/2016 Season/Split 2
	CBLOL/2016 Season/Split 2 Playoffs
	CBLOL/2017 Season/Split 1 Promotion
	CBLOL/2017 Season/Split 1
	CBLOL/2017 Season/Split 2 Promotion
	CBLOL/2017 Season/Split 1 Playoffs
	CBLOL/2017 Season/Split 2
	CBLOL/2017 Season/Split 2 Playoffs
	CBLOL/2018 Season/Split 1 Promotion
	CBLOL/2018 Season/Split 1
	CBLOL/2018 Season/Split 1 Playoffs
	CBLOL/2018 Season/Split 2 Promotion and Access
	CBLOL/2018 Season/Split 2
	CBLOL/2018 Season/Split 2 Play

In [ ]:
teams.loc[teams['team'].str.contains('BRION')]

In [ ]:
if len(names) > 0:
    teams = pd.concat(
        [
            teams,
            # pd.Series({'team': names.pop(0), 'team_id': get_new_id(teams)}).to_frame().T
            pd.Series({'team': names.pop(0), 'team_id': 8}).to_frame().T
        ],
        ignore_index=True
    )
names

In [ ]:
teams = teams.sort_values(by='team').reset_index(drop=True)
teams

In [ ]:
teams.to_csv('./csv/teams_id.csv', index=False)

## Players

In [330]:
players = pd.read_csv('./csv/players_id.csv')
names = {}

for league in leagues['League Short']:
    print(league)
    tournaments = get_tournaments(where=f'L.League_Short="{league}"')
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    for page in tournaments['OverviewPage']:
        print(f'\t{page}')
        try:
            scoreboard_players = get_scoreboard_players(
                where=f'T.OverviewPage="{page}"'
            )
            if scoreboard_players is None:
                continue
        except Exception as e:
            print(e)
            continue
        scoreboard_players = scoreboard_players.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        player_names = list(set(scoreboard_players[['Link', 'Team']].itertuples(index=False)))
        names = {}
        for (name, team) in player_names:
            if name not in players['player'].values:
                lst = names.get(team, [])
                lst.append(name)
                names[team] = lst
        if len(names) > 0:
            break
    if len(names) > 0:
        print(scoreboard_players['OverviewPage'].unique()[0])
        for team, player_lst in names.items():
            print(f'{team}\n  {player_lst}')
        break

2015 IWCT
	None
IWCQ
	2016 International Wildcard Qualifier
CBLOL
	CBLOL/2014 Season/Champions Series
	CBLOL/2014 Season/Champions Series/Playoffs
	CBLOL/2015 Season/Split 1 Qualifiers
	CBLOL/2015 Season/Split 1
	CBLOL/2015 Season/Split 1 Playoffs
	CBLOL/2015 Season/Split 2 Promotion
	CBLOL/2015 Season/Split 2
	CBLOL/2015 Season/Split 2 Playoffs
	CBLOL/2015 Season/Post-Season
	CBLOL/2016 Season/Split 1 Promotion
	CBLOL/2016 Season/Split 1
	CBLOL/2016 Season/Split 1 Playoffs
	CBLOL/2016 Season/Split 2 Promotion
	CBLOL/2016 Season/Split 2
	CBLOL/2016 Season/Split 2 Playoffs
	CBLOL/2017 Season/Split 1 Promotion
	CBLOL/2017 Season/Split 1
	CBLOL/2017 Season/Split 2 Promotion
	CBLOL/2017 Season/Split 1 Playoffs
	CBLOL/2017 Season/Split 2
CBLOL/2017 Season/Split 2
RED Canids
  ['rhuckz']
T Show
  ['Céos', 'Ceos', 'b4dd', 'Ferchu']
ProGaming Esports
  ['LUSKKA']
Keyd Stars
  ['Juzinho']
CNB e-Sports Club
  ['Lep']


In [331]:
teams = list(names.keys())
print(teams)
print()
print(teams[0])
print(names[teams[0]])

['RED Canids', 'T Show', 'ProGaming Esports', 'Keyd Stars', 'CNB e-Sports Club']

RED Canids
['rhuckz']


In [340]:
players.loc[players['player'].str.contains('lus', case=False)]

,player,player_id
59,Luskka,116


In [341]:
if len(names) > 0:
    name = names[teams[0]].pop(0)
    players = pd.concat(
        [
            players,
            # pd.Series({'player': name, 'player_id': get_new_id(players)}).to_frame().T
            pd.Series({'player': name, 'player_id': 116}).to_frame().T
        ],
        ignore_index=True
    )
    if len(names[teams[0]]) == 0:
        del names[teams[0]]
        del teams[0]
print(teams)
print()
if len(teams) > 0:
    print(teams[0])
    print(names[teams[0]])

['Keyd Stars', 'CNB e-Sports Club']

Keyd Stars
['Juzinho']


In [328]:
players = players.sort_values(by='player').reset_index(drop=True)
players

,player,player_id
0,4LaN,132
1,7ico,77
2,Absolut,141
3,Alocs,69
4,Anjinho,75
...,...,...
150,tinowns,50
151,tockers,41
152,verfix,114
153,w0lv,43


In [329]:
players.to_csv('./csv/players_id.csv', index=False)

## Scoreboard

In [ ]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

In [ ]:
# for year in range(2011, 2024):
for year in range(2023, 2024):
    tournaments = pd.DataFrame()
    # for league in TARGET_LEAGUES:
    for league in leagues['League Short']:
        t = get_tournaments(where=f'L.League_Short="{league}" and T.Year={year}')
        tournaments = pd.concat([tournaments, t])
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(year, tournaments.shape)

In [ ]:
# for year in range(2011, 2024):
for year in range(2023, 2024):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    print(year, tournaments.shape)
    scoreboard_games = pd.DataFrame()
    for page in tournaments['OverviewPage']:
        sg = get_scoreboard_games(where=f'T.OverviewPage="{page}"')
        if sg is None:
            print(f'{year}\t{page:35}\tdrop')
            tournaments.drop(tournaments.loc[tournaments['OverviewPage'] == page].index, inplace=True)
            continue
        league = tournaments.loc[tournaments['OverviewPage'] == page, 'League'].iloc[0]
        league = leagues.loc[leagues['League'] == league, 'League Short'].iloc[0]
        sg['League'] = league
        sg = sg.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        print(f'\t{sg.shape}\t{page}')
        scoreboard_games = pd.concat([scoreboard_games, sg])
    scoreboard_games.to_csv(f'./csv/scoreboard_games/{year}_scoreboard_games.csv', index=False)
    print(f'{year} scoreboard_games {scoreboard_games.shape}')
    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(f'{year} tournaments {tournaments.shape}')

In [ ]:
scoreboard_games